In [75]:
import numpy as np
import plotly.express as px
import pandas as pd

def trajAffine(traj,scale,c2w):
    traj = np.vstack([scale*traj, np.ones(traj.shape[1])])
    traj = (c2w @ traj).T
    traj = traj[:,:-1]
    return traj

In [62]:
data_folder = "drawing/data/"
test_df = pd.read_pickle(f"../{data_folder}flight_test.pkl")
test_df_human = test_df[~test_df["2d_human"].isna()]

In [64]:
t =np.stack(test_df_human["3d_gt"])

In [52]:
print(np.min(t[:,-1,1]))
np.argmin(t[:,-1,1])


-1.0381944599688318


np.int64(103)

In [89]:
k = test_df_human.loc[0, "2d_human"]
exx = np.array([0, 1280])
exy = np.array([0,720])
fig = px.line(x=k[:,0], y=k[:,1])
fig.add_trace(px.scatter(x=exx, y=exy, opacity=0).data[0])
fig.update_yaxes(autorange='reversed')

In [3]:
pos = np.loadtxt("pos.txt")
quat = np.loadtxt("quat.txt")

In [54]:
from scipy.spatial.transform import Rotation as R
whichpoint = 30
r = R.from_quat(quat[whichpoint], scalar_first=True)

dlf_ocv = np.array([[0,0,1,0],
                    [1,0,0,0],
                    [0,1,0,0],
                    [0,0,0,1]])

fli_dlf = np.identity(4)
fli_dlf[:3, :3] =  R.as_matrix(r)
fli_dlf[:3,-1] = pos[whichpoint]


tr = r.as_rotvec()
tr /= np.linalg.norm(tr)

xv,yv,zv = list(r.as_matrix().T)

p0 = pos[whichpoint]
p1 = pos[whichpoint] + xv
p = np.stack([p0,p1])

In [55]:
realg = np.stack([(r.as_matrix().T + pos[whichpoint]),(0*r.as_matrix().T + pos[whichpoint])])

In [ ]:
def pose_to_fli_ocv(pos, quat):
    dlf_ocv = np.array([[0,0,1,0],
                        [1,0,0,0],
                        [0,1,0,0],
                        [0,0,0,1]])
    # Given xyz and quaternion, that takes vectors from the ned drone local frame to the ned flightroom frame
    r = R.from_quat(quat, scalar_first=True)
    fli_dlf = np.identity(4)
    fli_dlf[:3, :3] =  R.as_matrix(r)
    fli_dlf[:3,-1] = pos
    return fli_dlf @ dlf_ocv

In [ ]:
T = pose_to_fli_osv(pos[whichpoint], quat[whichpoint]) @ dlf_ocv

In [90]:
traj = t[0].T
retraj = trajAffine(traj, 1.0, T)

In [93]:
retraj.shape

(100, 3)

In [88]:
# might have to flip Y when going from MOCAP to NED?
ex = np.array([np.min(pos),np.max(pos)])
fig = px.scatter_3d(x=pos[:,0], y=pos[:,1], z=pos[:,2], color=np.arange(len(pos))).update_traces(marker=dict(size=1))
fig.add_trace(px.scatter_3d(x=ex, y=ex, z=ex, opacity=0).data[0])
fig.add_trace(px.line_3d(x=realg[:,0,0], y=realg[:,0,1], z=realg[:,0,2]).update_traces(line=dict(color='red')).data[0])
fig.add_trace(px.line_3d(x=realg[:,1,0], y=realg[:,1,1], z=realg[:,1,2]).update_traces(line=dict(color='blue')).data[0])
fig.add_trace(px.line_3d(x=realg[:,2,0], y=realg[:,2,1], z=realg[:,2,2]).update_traces(line=dict(color='green')).data[0])
fig.add_trace(px.line_3d(x=retraj[:,0], y=retraj[:,1], z=retraj[:,2]).update_traces(line=dict(color='cyan')).data[0])
fig.update_layout(scene=dict(zaxis=dict(autorange='reversed')))
fig.update_layout(scene=dict(yaxis=dict(autorange='reversed')))